### Data preparations

In [2]:
!pip install lightning
!pip install pytorchvideo

In [3]:
from pytorchvideo.data import LabeledVideoDataset, make_clip_sampler, labeled_video_dataset
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    UniformTemporalSubsample,
    Permute
)
from torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip,
    Resize
)
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo
)

/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [4]:
video_transforms = Compose([
    ApplyTransformToKey(key='video',
    transform=Compose([
        UniformTemporalSubsample(20),
        Normalize((0.45, 0.45, 0.45),(0.225, 0.225, 0.225)),
        # RandomShortSideScale(min_size=248, max_size=256),
        # CenterCropVideo(224),
        RandomHorizontalFlip(p=0.5),
    ]),
    ),
])

In [5]:
import pandas as pd
import numpy as np
import os
import shutil

In [6]:
from torch.utils.data import DataLoader

In [7]:
dataset_path = "/kaggle/input/cells-dataset/Cells classification/dataset"

In [8]:
os.chdir(dataset_path)
os.chdir("..")

In [9]:
target = 'erythrocytes'

In [10]:
def get_value_from_normalized(norm_value, target_min, target_max):
    return norm_value * (target_max - target_min) + target_min

In [11]:
dataframe = pd.read_csv("DataFrame.csv", index_col=0)

In [12]:
target_min = dataframe[target].min()
target_max = dataframe[target].max()
dataframe[target] = (dataframe[target] - target_min) / (target_max - target_min)

In [13]:
dataframe = dataframe.sample(frac=1)
ratio = 0.8
train_size = int(dataframe.shape[0] * ratio)
train_data = dataframe[0:train_size]
test_data = dataframe[train_size:]
train_data, test_data

(          files  erythrocytes  lymphocytes  high_erythrocytes  \
 689  traffic690      0.000000           10                  0   
 493  traffic494      0.291667           14                  0   
 115  traffic116      0.541667           10                  1   
 859  traffic860      0.500000           17                  1   
 811  traffic812      0.458333           10                  1   
 ..          ...           ...          ...                ...   
 286  traffic287      0.291667           15                  0   
 585  traffic586      0.208333           14                  0   
 997  traffic998      0.208333            3                  0   
 669  traffic670      0.250000           13                  0   
 120  traffic121      0.375000           10                  0   
 
      high_lymphocytes  blur  noise  
 689                 0     0      0  
 493                 1     0      0  
 115                 0     0      0  
 859                 1     0      0  
 811            

In [14]:
class CustomDataset(LabeledVideoDataset):
    def __init__(self, dataset_path, dataframe, target_name, transforms, clip_sampler_type='random', clip_duration=1):
      df = dataframe.reset_index()
      paths = []
      for i, file_name in enumerate(df['files']):
          temp_dict = df.iloc[i].to_dict()
          temp_dict['label'] = df[target_name][i]
          temp_dict.pop(target_name)
          temp_dict.pop('files')
          temp_dict.pop('index')
          paths.append((f"{dataset_path}/{file_name}", temp_dict))
      super().__init__(labeled_video_paths=paths,
                       clip_sampler=make_clip_sampler(clip_sampler_type, clip_duration),
                       transform=transforms, decode_audio=False)

In [15]:
train_dataset = CustomDataset(dataset_path=dataset_path, dataframe=train_data,
                              target_name=target,
                              transforms=video_transforms)

In [16]:
train_dataset.num_videos

800

In [44]:
test_value = next(iter(train_dataset))

In [48]:
test_value['label']

0.5416666666666666

In [46]:
get_value_from_normalized(test_value["label"], target_min, target_max)

13.0

### Model

In [17]:
import torch
import torch.nn as nn
from pytorch_lightning import LightningModule, seed_everything, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.metrics import classification_report
import torchmetrics

In [18]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [19]:
class TestModelApproximation(LightningModule):
    def __init__(self, target_label):
        super(TestModelApproximation, self).__init__()
        # model architecture
        self.video_model = torch.hub.load("facebookresearch/pytorchvideo", "efficient_x3d_xs", pretrained=True)
        self.relu = nn.ReLU()
        self.linear = nn.Linear(400, 1)

        self.lr = 1e-3
        self.batch_size = 8
        self.numworkers = 0
        # evaluation metric
        self.metric = torchmetrics.MeanSquaredError()
        # loss function
        self.criterion = nn.MSELoss()
        # helpers
        self.target_label = target_label
        self.training_step_outputs = []
        self.validation_step_outputs = []
        self.testing_step_outputs = []

    def forward(self, x):
        x = self.video_model(x)
        x = self.relu(x)
        x = self.linear(x)
        return x

    def configure_optimizers(self):
        opt = torch.optim.AdamW(params=self.parameters(), lr=self.lr)
        scheduler = CosineAnnealingLR(opt, T_max=10, eta_min=1e-6, last_epoch=-1)
        return {'optimizer': opt, 'lr_scheduler': scheduler}

    def train_dataloader(self):
        dataset = CustomDataset(dataset_path=dataset_path, dataframe=train_data,
                              target_name=self.target_label,
                              transforms=video_transforms)
        loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.numworkers, pin_memory=True)
        return loader

    def training_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        out = torch.reshape(self.forward(video), (8,)).to(torch.double)
        loss = self.criterion(out, label)
        metric = self.metric(out, label.to(torch.int64))
        self.training_step_outputs.append({'loss': loss, 'metric': metric})
        return {'loss': loss, 'metric': metric}

    def on_train_epoch_end(self):
        outputs = self.training_step_outputs
        loss = torch.stack([x['loss'] for x in outputs]).mean().cpu().detach().numpy().round(2)
        metric = torch.stack([x['metric'] for x in outputs]).mean().cpu().detach().numpy().round(2)
        self.log('train_loss', loss)
        self.log('train_metric', metric)

    def val_dataloader(self):
        dataset = CustomDataset(dataset_path=dataset_path, dataframe=test_data,
                              target_name=self.target_label,
                              transforms=video_transforms)
        loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.numworkers, pin_memory=True)
        return loader

    def validation_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        out = torch.reshape(self.forward(video), (8,)).to(torch.double)
        loss = self.criterion(out, label)
        metric = self.metric(out, label.to(torch.int64))
        self.validation_step_outputs.append({'loss': loss, 'metric': metric})
        return {'loss': loss, 'metric': metric}

    def on_validation_epoch_end(self):
        outputs = self.validation_step_outputs
        loss = torch.stack([x['loss'] for x in outputs]).mean().cpu().detach().numpy().round(2)
        metric = torch.stack([x['metric'] for x in outputs]).mean().cpu().detach().numpy().round(2)
        self.log('val_loss', loss)
        self.log('val_metric', metric)

    def test_dataloader(self):
        dataset = CustomDataset(dataset_path=dataset_path, dataframe=test_data,
                              target_name=self.target_label,
                              transforms=video_transforms)
        loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.numworkers, pin_memory=True)
        return loader

    def test_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        out = torch.reshape(self.forward(video), (8,)).to(torch.double)
        self.testing_step_outputs.append({'label': label, 'pred': out})
        return {'label': label, 'pred': out}

    def on_test_epoch_end(self):
        outputs = self.testing_step_outputs
        label = torch.cat([x['label'] for x in outputs]).cpu().detach().numpy()
        pred = torch.cat([x['pred'] for x in outputs]).cpu().detach().numpy()
        print(f"MSE: {mean_squared_error(label, pred)}\nMAE: {mean_absolute_error(label, pred)}\nR^2: {r2_score(label, pred)}")

In [20]:
os.chdir("/kaggle/working/")

In [21]:
checkpoint_callback = ModelCheckpoint(monitor="val_loss", dirpath="checkpoints", filename="file", save_last=True)
lr_monitor = LearningRateMonitor(logging_interval="epoch")
model = TestModelApproximation(target_label=target)
seed_everything(0)
trainer = Trainer(max_epochs=100,
                  precision=16,
                  accumulate_grad_batches=2,
                  enable_progress_bar=True,
                  num_sanity_val_steps=0,
                  callbacks=[lr_monitor, checkpoint_callback])

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main
/opt/conda/lib/python3.10/site-packages/lightning_fabric/connector.py:554: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(


In [22]:
trainer.fit(model)

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:617: UserWarning: Checkpoint directory /kaggle/working/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [23]:
trainer.validate(model)

Validation: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │            0.0            │
│        val_metric         │    0.20000000298023224    │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 0.0, 'val_metric': 0.20000000298023224}]

In [24]:
trainer.test(model)

Testing: 0it [00:00, ?it/s]

MSE: 0.003428589758358511
MAE: 0.04036548932393392
R^2: 0.9036231485467104


[{}]